In [1]:
from PIL import Image
from pathlib import Path
import numpy as np

In [2]:
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions

In [3]:
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Resizing

In [6]:
EXPECTED_SIZE = (224, 224)

In [7]:
GIRLS = ('noam', 'shira', 'other')

In [8]:
def load_img(img_path:Path):
    img = Image.open(img_path)
    img = img.resize(EXPECTED_SIZE)
    img_array = np.asarray(img, dtype=np.float32)
#     img_norm = img_array / 255.0 # scale uint8 coded colors to the [0.0, 1.0] floats
    return img_array

In [9]:
images_dir = Path("../../images/faces/")
image_paths = list(images_dir.glob("*/*jpg"))

In [11]:
image_paths[0].parent.name

'noam'

In [12]:
faces = np.zeros((len(image_paths), *EXPECTED_SIZE, 3), dtype=np.float32)
target_label_idxs = []

for i, img_path in enumerate(image_paths):
    face = load_img(img_path)
    girl_name = img_path.parent.name
    girl_name_idx = GIRLS.index(girl_name)
    
    faces[i, ...] = face
    target_label_idxs.append(girl_name_idx)

In [13]:
faces_ready_for_model = preprocess_input(faces, version=2)
face_labels = to_categorical(target_label_idxs)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(faces_ready_for_model, face_labels, train_size=0.8, stratify=face_labels, random_state=0)

In [15]:
base_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
 
model = Sequential()
model.add(Resizing(224, 224))
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(len(GIRLS), activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

2022-07-30 11:24:14.733784: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
hist = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=10, epochs=3)

Epoch 1/3
12/12 [==============================] - 8s 454ms/step - loss: 6.9998 - accuracy: 0.5982 - val_loss: 1.5418 - val_accuracy: 0.8276
Epoch 2/3
12/12 [==============================] - 5s 383ms/step - loss: 0.3381 - accuracy: 0.9464 - val_loss: 0.1531 - val_accuracy: 0.9655
Epoch 3/3
12/12 [==============================] - 4s 371ms/step - loss: 0.1019 - accuracy: 0.9732 - val_loss: 0.3891 - val_accuracy: 0.9655


In [17]:
model.save("trained_vgg_on_girls")

INFO:tensorflow:Assets written to: trained_vgg_on_girls/assets


INFO:tensorflow:Assets written to: trained_vgg_on_girls/assets


In [18]:
samples = np.zeros((2, *EXPECTED_SIZE, 3), dtype=np.float32)
shira = load_img("../shirush.jpeg")
noam = load_img("../noamush.jpeg")
samples[0, ...] = shira
samples[1, ...] = noam

In [19]:
samples = preprocess_input(samples, version=2)
result = model.predict(samples)

1/1 [==============================] - 1s 666ms/step


In [20]:
result

array([[2.3645646e-26, 1.0000000e+00, 9.8785807e-19],
       [1.0000000e+00, 0.0000000e+00, 2.2672887e-22]], dtype=float32)